In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [69]:
import src.load as load
import app.utils as utils
import numpy as np
import pandas as pd
import src.table as table
import src.graphs as graphs
import plotly.graph_objects as go
train_dataset, test_dataset, mappings, reverse_mappings, train_meta, test_meta  = load.separate_year('../data/processed/')

models = utils.load_models('../app/models')

In [46]:
actuals = test_dataset.Y_kilos.numpy()

In [39]:
folder_path = '../app/preds'

In [3]:
transplant_weeks = test_meta.WeekTransplanted.values

In [41]:
utils.populate_preds(folder_path,models,transplant_weeks,test_dataset)

In [57]:
kilo_preds, sched_preds = utils.read_preds('../app/preds',test_dataset)

In [58]:
kilo_preds.shape, sched_preds.shape

((176, 52, 40), (176, 52, 5))

In [59]:
season_preds, season_actuals = table.season_shift(transplant_weeks,kilo_preds,actuals)

In [60]:
season_preds.shape

(176, 52, 71)

In [61]:
season_actuals

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [64]:
summed_preds = np.sum(kilo_preds, axis=0)  # Shape: (20, 51)
summed_actuals = np.sum(actuals, axis=0) 

In [66]:
summed_preds.shape

(52, 40)

In [67]:
summed_actuals.shape

(40,)

In [68]:
max_pred_value = np.max(summed_preds)

In [112]:
traces = []
for i in range(summed_preds.shape[0]):  # Iterate over the 20 middle dimension
    trace = go.Scatter(
        x=np.arange(summed_preds.shape[1]),  # X-axis: 0 to 50
        y=summed_preds[i, :],  # Y-axis: summed predictions for each middle dimension
        mode='lines',
        name=f'Preds Trace {i+1}',
        visible=(i == 0)  # Only the first trace is visible initially
    )
    traces.append(trace)

In [123]:
actuals_trace = go.Bar(
    x=np.arange(summed_actuals.shape[0]),  # X-axis: 0 to 50
    y=summed_actuals,  # Y-axis: summed actuals
    name='Actuals',
    visible=True
)

In [115]:
traces.append(actuals_trace)

In [118]:
fig = go.Figure(data=traces)

In [119]:
steps = []
for i in range(len(traces) - 1):  # Exclude the actuals trace from the slider
    step = dict(
        method='update',
        args=[{'visible': [j == i for j in range(len(traces) - 1)] + [True]}],  # Show only one preds trace and the actuals
        label=f'Preds Trace {i+1}'
    )
    steps.append(step)

In [120]:
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Trace: "},
    pad={"t": summed_preds.shape[1]},
    steps=steps
)]

In [122]:
fig.update_layout(
        title='Predictions and Actuals',
        xaxis_title='Time Step',
        yaxis_title='Values',
        barmode='overlay',
        sliders=sliders,
        yaxis=dict(range=[0, max_pred_value])  # Set the y-axis scale based on the max of preds
    )